In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import cv2
import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

import keras
from keras import layers

import tensorflow as tf

In [ ]:
# Các thông số
IMAGE_SIZE = 640
BATCH_SIZE = 16
# Hàm load và tiền xử lý ảnh
def load_data(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = image / 255.0
    return image

# Hàm tạo dataset
def data_generator(image_paths):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [ ]:

def plot_results(images, titles, figure_size=(12, 12)):
    fig = plt.figure(figsize=figure_size)
    for i in range(len(images)):
        fig.add_subplot(1, len(images), i + 1).set_title(titles[i])
        _ = plt.imshow(images[i])
        plt.axis("off")
    plt.show()

"""
def infer(original_image):
    image = keras.utils.img_to_array(original_image)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    output_image = zero_dce_model(image)
    output_image = tf.cast((output_image[0, :, :, :] * 255), dtype=np.uint8)
    output_image = Image.fromarray(output_image.numpy())
    return output_image
"""

'\ndef infer(original_image):\n    image = keras.utils.img_to_array(original_image)\n    image = image.astype("float32") / 255.0\n    image = np.expand_dims(image, axis=0)\n    output_image = zero_dce_model(image)\n    output_image = tf.cast((output_image[0, :, :, :] * 255), dtype=np.uint8)\n    output_image = Image.fromarray(output_image.numpy())\n    return output_image\n'

In [ ]:
def denoise_image(image):
    # Convert image to Lab color space for better color preservation
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    # Apply Non-Local Means Denoising on the L channel (lightness)
    l_denoised = cv2.fastNlMeansDenoising(l, None, h=10, templateWindowSize=7, searchWindowSize=21)
    # Ensure all channels have the same size (if required, resize a and b to match l_denoised)
    a = cv2.resize(a, (l_denoised.shape[1], l_denoised.shape[0]))
    b = cv2.resize(b, (l_denoised.shape[1], l_denoised.shape[0]))

    # Merge back the channels
    lab_denoised = cv2.merge((l_denoised, a, b))
    denoised_image = cv2.cvtColor(lab_denoised, cv2.COLOR_Lab2BGR)
    return denoised_image


In [ ]:

def denoise_image(image):
    # Convert image to Lab color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)

    # Giảm h xuống 7 thay vì 10
    l_denoised = cv2.fastNlMeansDenoising(l, None, h=5,
                                          templateWindowSize=7,
                                          searchWindowSize=21)

    # Resize a và b nếu cần
    a = cv2.resize(a, (l_denoised.shape[1], l_denoised.shape[0]))
    b = cv2.resize(b, (l_denoised.shape[1], l_denoised.shape[0]))

    # Merge lại
    lab_denoised = cv2.merge((l_denoised, a, b))
    denoised_image = cv2.cvtColor(lab_denoised, cv2.COLOR_Lab2BGR)
    return denoised_image

def sharpen_image(image):
    # Khử nhiễu trước
    denoised_image = denoise_image(image)

    # Tạo ảnh mờ
    blurred_image = cv2.GaussianBlur(denoised_image, (3, 3), 1.0)

    # Unsharp Mask (điều chỉnh alpha, beta)
    # alpha = 1.2, beta = -0.2, có thể tăng/giảm thêm
    unsharp_mask = cv2.addWeighted(denoised_image, 1.2,
                                   blurred_image, -0.2, 0)

    # Đảm bảo giá trị không vượt [0..255]
    sharpened_image = np.clip(unsharp_mask, 0, 255).astype(np.uint8)

    return sharpened_image


In [ ]:
# Đường dẫn tới thư mục chứa ảnh
Dataset_path = "/content/drive/MyDrive/CS406/Enhanced_Dataset/train/images"
test_images = []

# Lấy danh sách tất cả ảnh trong thư mục "image"
if os.path.isdir(Dataset_path):  # Kiểm tra xem có phải thư mục không
    images_path = [os.path.join(Dataset_path, img) for img in os.listdir(Dataset_path)]
    split_index = 30  # Số lượng ảnh để lấy cho test
    test_images.extend(images_path[:split_index])

print("Số lượng ảnh test:", len(test_images))

# Tạo dataset từ danh sách ảnh test
test_dataset = data_generator(test_images)

# Kiểm tra dataset
for batch in test_dataset.take(6):  # Lấy batch đầu tiên để xem thử
    print("Kích thước của batch:", batch.shape)

Số lượng ảnh test: 30
Kích thước của batch: (16, 640, 640, 3)


In [ ]:

for val_image_file in test_images:
    original_image = Image.open(val_image_file)
    enhanced_image_1 = denoise_image(np.array(original_image))
    enhanced_image_2 = sharpen_image(denoise_image(np.array(original_image)))
    plot_results(
        [original_image, enhanced_image_1, enhanced_image_2],
        ["Original", "enhanced_image_1", "enhanced_image_2"],
        (20, 12),
    )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import os
import random
from PIL import Image
import numpy as np

# Đường dẫn dataset gốc và dataset tăng cường
Dataset_path = "/content/drive/MyDrive/CS406/Enhanced_Dataset"
Enhanced_Dataset_path = "/content/drive/MyDrive/CS406/Enhanced_Dataset_And_Denoise"
new_size = 640  # Kích thước mới của ảnh

# Tạo cấu trúc thư mục mới (train, val, test / images, labels)
sets = ["train", "val", "test"]
for set_name in sets:
    os.makedirs(os.path.join(Enhanced_Dataset_path, set_name, "images"), exist_ok=True)
    os.makedirs(os.path.join(Enhanced_Dataset_path, set_name, "labels"), exist_ok=True)

# Duyệt qua từng bộ dữ liệu (train, val, test)
for set_name in sets:
    image_folder = os.path.join(Dataset_path, set_name, "images")
    label_folder = os.path.join(Dataset_path, set_name, "labels")

    # Lấy danh sách ảnh
    images = [img for img in os.listdir(image_folder)]

    for img_name in images:
        img_path = os.path.join(image_folder, img_name)
        label_name = img_name.rsplit('.', 1)[0] + ".txt"  # Tên nhãn tương ứng với ảnh
        label_path = os.path.join(label_folder, label_name)

        # Kiểm tra xem nhãn có tồn tại không
        if os.path.exists(label_path):
            # Tăng cường ảnh
            original_image = Image.open(img_path).convert("RGB").resize((new_size, new_size))
            enhanced_image = sharpen_image(denoise_image(np.array(original_image)))
            enhanced_image = Image.fromarray(enhanced_image)
            # Lưu ảnh đã tăng cường vào thư mục mới
            enhanced_image.save(os.path.join(Enhanced_Dataset_path, set_name, "images", img_name))
            """
            # Sao chép nhãn vào thư mục mới
            with open(label_path, 'r') as label_file:
                lines = label_file.readlines()
            with open(os.path.join(Enhanced_Dataset_path, set_name, "labels", label_name), 'w') as new_label_file:
                new_label_file.writelines(lines)
            """
print("Hoàn thành xử lý và lưu ảnh và nhãn vào cấu trúc thư mục mới.")


Hoàn thành xử lý và lưu ảnh và nhãn vào cấu trúc thư mục mới.
